### m8.1stan

m8.1stan is the first model in the Statistical Rethinking book (pp. 249) using Stan.

Here we will use Turing's NUTS support, which is currently (2018) the originalNUTS by [Hoffman & Gelman]( http://www.stat.columbia.edu/~gelman/research/published/nuts.pdf) and not the one that's in Stan 2.18.2, i.e., Appendix A.5 in: https://arxiv.org/abs/1701.02434

The StatisticalRethinking pkg imports modules such as CSV and DataFrames

In [1]:
using StatisticalRethinking, Turing

Turing.setadbackend(:reverse_diff);
Turing.turnprogress(false);

loaded


┌ Info: [Turing]: global PROGRESS is set as false
└ @ Turing /Users/rob/.julia/packages/Turing/xp88X/src/Turing.jl:81


Read in the `rugged` data as a DataFrame

In [2]:
d = CSV.read(rel_path("..", "data", "rugged.csv"), delim=';');

Show size of the DataFrame (should be 234x51)

In [3]:
size(d)

(234, 51)

Apply log() to each element in rgdppc_2000 column and add it as a new column

In [4]:
d = hcat(d, map(log, d[Symbol("rgdppc_2000")]));

Rename our col x1 => log_gdp

In [5]:
rename!(d, :x1 => :log_gdp);

Now we need to drop every row where rgdppc_2000 == missing

When this (https://github.com/JuliaData/DataFrames.jl/pull/1546) hits DataFrame it'll be conceptually easier: i.e., completecases!(d, :rgdppc_2000)

In [6]:
notisnan(e) = !ismissing(e)
dd = d[map(notisnan, d[:rgdppc_2000]), :];

Updated DataFrame dd size (should equal 170 x 52)

In [7]:
size(dd)

(170, 52)

Define the Turing model

In [8]:
@model m8_1stan(y, x₁, x₂) = begin
    σ ~ Truncated(Cauchy(0, 2), 0, Inf)
    βR ~ Normal(0, 10)
    βA ~ Normal(0, 10)
    βAR ~ Normal(0, 10)
    α ~ Normal(0, 100)

    for i ∈ 1:length(y)
        y[i] ~ Normal(α + βR * x₁[i] + βA * x₂[i] + βAR * x₁[i] * x₂[i], σ)
    end
end;

Test to see that the model is sane. Use 2000 for now, as in the book.
Need to set the same stepsize and adapt_delta as in Stan...

Use Turing mcmc

In [9]:
posterior = sample(m8_1stan(dd[:log_gdp], dd[:rugged], dd[:cont_africa]),
    Turing.NUTS(2000, 200, 0.95));
# Describe the posterior samples
describe(posterior)

┌ Info: [Turing] looking for good initial eps...
└ @ Turing /Users/rob/.julia/packages/Turing/xp88X/src/samplers/support/hmc_core.jl:246
[NUTS{Turing.FluxTrackerAD,Union{}}] found initial ϵ: 0.05
└ @ Turing /Users/rob/.julia/packages/Turing/xp88X/src/samplers/support/hmc_core.jl:291
┌ Info:  Adapted ϵ = 0.02494112896959952, std = [1.0, 1.0, 1.0, 1.0, 1.0]; 200 iterations is used for adaption.
└ @ Turing /Users/rob/.julia/packages/Turing/xp88X/src/samplers/adapt/adapt.jl:91


[NUTS] Finished with
  Running time        = 284.69474977300035;
  #lf / sample        = 0.0015;
  #evals / sample     = 47.82;
  pre-cond. metric    = [1.0, 1.0, 1.0, 1.0, 1.0].
Iterations = 1:2000
Thinning interval = 1
Chains = 1
Samples per chain = 2000

Empirical Posterior Estimates:
              Mean          SD         Naive SE       MCSE         ESS   
       α    9.20431457  0.409119788 0.00914819658 0.0277999222  216.57774
  lf_num    0.00150000  0.067082039 0.00150000000 0.0015000000 2000.00000
      βA   -1.92760777  0.347059686 0.00776049050 0.0286062609  147.19266
      βR   -0.20010279  0.111203536 0.00248658666 0.0076298260  212.42612
       σ    0.96306823  0.240119729 0.00536924037 0.0150168225  255.68167
 elapsed    0.14234737  0.166759748 0.00372886132 0.0080236235  431.95785
 epsilon    0.02569079  0.013752776 0.00030752142 0.0005170151  707.57793
eval_num   47.82000000 26.804754194 0.59937252499 0.6517587542 1691.41396
     βAR    0.39846959  0.186980959 0.0041810

Example of a Turing run simulation output

Here's the ulam() output from rethinking (note that in above output the SD value is too large).

In [10]:
m81rethinking = "
       Mean StdDev lower 0.89 upper 0.89 n_eff Rhat
 a      9.24   0.14       9.03       9.47   291    1
 bR    -0.21   0.08      -0.32      -0.07   306    1
 bA    -1.97   0.23      -2.31      -1.58   351    1
 bAR    0.40   0.13       0.20       0.63   350    1
 sigma  0.95   0.05       0.86       1.03   566    1
";#-

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*